<a href="https://colab.research.google.com/github/dhamu2908/DeepLearningAssignment2/blob/main/DeepLearningAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Deep Learning Assignment 2
CS24M027 Dhamodharan Muthu Muniyandi
IIT MADRAS
"""

'\nDeep Learning Assignment 2\nCS24M027 Dhamodharan Muthu Muniyandi\nIIT MADRAS\n'

In [ ]:
import os

#Data Accessing check

print("Exploring contents of /kaggle/input/dldata:")
dataset_path = "/kaggle/input/dldata/inaturalist_12K"
train_path = os.path.join(dataset_path, "train")
val_path = os.path.join(dataset_path, "val")

# Verifying the existence of directories
is_train_available = os.path.isdir(train_path)
is_val_available = os.path.isdir(val_path)


print(f"Train directory found: {is_train_available}")
print(f"Validation directory found: {is_val_available}")

if is_train_available:
    print("Sample items in training folder:", os.listdir(train_path)[:5])

if is_val_available:
    print("Sample items in validation folder:", os.listdir(val_path)[:5])

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import time
import wandb

In [ ]:
# Initialize WandB
wandb.login(key = "" )

In [ ]:
# Define training configuration parameters
config = {
    "num_epochs": 10,
    "lr": 1e-4,
    "batch_sz": 32,
    "backbone": "resnet50",
    "unfreeze_layers": 1
}

IMG_DIM = 224
NUM_CLASSES = 10

# Function to divide dataset into training and validation sets while preserving class balance
def stratified_split(dataset, train_frac):
    train_ids = []
    val_ids = []

    # Index ranges for each class (based on specific dataset arrangement)
    class_boundaries = [
        (0, 999), (1000, 1999), (2000, 2999), (3000, 3999), (4000, 4998),
        (4999, 5998), (5999, 6998), (6999, 7998), (7999, 8998), (8999, 9998)
    ]

    for lower, upper in class_boundaries:
        indices = list(range(lower, upper + 1))
        split_point = int(len(indices) * train_frac)
        train_ids.extend(indices[:split_point])
        val_ids.extend(indices[split_point:])

    training_data = Subset(dataset, train_ids)
    validation_data = Subset(dataset, val_ids)

    return training_data, validation_data


In [ ]:
# Responsible for creating data loaders and returning dataset statistics
def build_data_pipeline(config):
    size = (IMAGE_DIMENSION, IMAGE_DIMENSION)

    def get_transform():
        return transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor()
        ])

    # Define paths for training and testing datasets
    path_train = "/kaggle/input/nature1/inaturalist_12K/train"
    path_test = "/kaggle/input/nature1/inaturalist_12K/val"

    # Load raw datasets
    complete_train_dataset = ImageFolder(root=path_train, transform=get_transform())
    test_dataset = ImageFolder(root=path_test, transform=get_transform())

    # Split training set into train and validation sets
    splitter = DatasetSplitter(complete_train_dataset, 0.8)
    train_subset, val_subset = splitter.perform_split()

    # Create data loaders
    bs = config["batch_size"]
    loader_train = DataLoader(train_subset, batch_size=bs, shuffle=True)
    loader_val = DataLoader(val_subset, batch_size=bs, shuffle=True)
    loader_test = DataLoader(test_dataset, batch_size=bs, shuffle=True)

    # Return everything in a structured dictionary
    return {
        "train_size": len(train_subset),
        "val_size": len(val_subset),
        "test_size": len(test_dataset),
        "train_loader": loader_train,
        "val_loader": loader_val,
        "test_loader": loader_test
    }


In [ ]:
# Constructs and customizes a ResNet-50 model based on the provided configuration
def build_resnet_model(config):
    from torchvision.models import resnet50

    # Load pretrained ResNet-50 architecture
    resnet = resnet50(weights="IMAGENET1K_V1")
    input_features = resnet.fc.in_features

    # Replace the final layer to match the number of output classes
    resnet.fc = torch.nn.Linear(input_features, TOTAL_CLASSES)

    # Initially freeze all layers
    for param in resnet.parameters():
        param.requires_grad = False

    # Selectively unfreeze last 'k' layers
    unfreeze_count = config["unfreeze_layers"]
    if unfreeze_count > 0:
        for param in list(resnet.parameters())[-unfreeze_count:]:
            param.requires_grad = True

    return resnet


In [ ]:
# Trains a model using the provided configuration and dataset, logs metrics to Weights & Biases
def run_training(config, dataset_bundle):
    import wandb

    # Set device context
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Model setup
    model = build_resnet_model(config)
    model = torch.nn.DataParallel(model, device_ids=[0]).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

    # Load training and validation data
    train_loader = dataset_bundle["train_loader"]
    val_loader = dataset_bundle["val_loader"]
    n_train = dataset_bundle["train_size"]
    n_val = dataset_bundle["val_size"]

    for ep in range(config["num_epochs"]):
        model.train()
        train_loss_accumulator = 0.0
        correct_train_preds = 0

        for step, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

            train_loss_accumulator += loss.item()
            correct_train_preds += (logits.argmax(dim=1) == y_batch).sum().item()

            if step % 10 == 0:
                batch_acc = (logits.argmax(1) == y_batch).float().mean().item()
                print(f"[Epoch {ep} | Batch {step}] Accuracy: {batch_acc:.4f}, Loss: {loss.item():.4f}")

        # Evaluate on validation set
        model.eval()
        val_loss_accumulator = 0.0
        correct_val_preds = 0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                val_logits = model(x_val)
                loss = criterion(val_logits, y_val)
                val_loss_accumulator += loss.item()
                correct_val_preds += (val_logits.argmax(1) == y_val).sum().item()

        # Calculate and log metrics
        train_accuracy = correct_train_preds / n_train
        train_loss = train_loss_accumulator / len(train_loader)
        val_accuracy = correct_val_preds / n_val
        val_loss = val_loss_accumulator / len(val_loader)

        print(f"Epoch {ep} | Train Acc: {train_accuracy:.4f} | Train Loss: {train_loss:.4f} | Val Acc: {val_accuracy:.4f} | Val Loss: {val_loss:.4f}")
        wandb.log({
            "epoch": ep,
            "train_accuracy": train_accuracy,
            "train_loss": train_loss,
            "val_accuracy": val_accuracy,
            "val_loss": val_loss
        })

    print("Training complete.")
    torch.save(model.state_dict(), "./model.pth")
